In [1]:
import getpass
import os

# Force re-entering the API key
os.environ.pop("GROQ_API_KEY", None)
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [2]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    model='deepseek-r1-distill-llama-70b',
    temperature=0.1,
    api_key=os.environ["GROQ_API_KEY"]
)

response = llm.invoke("What is the capital of France?")

print(response.content)

<think>

</think>

The capital of France is Paris.


### Web Scraping


In [4]:
%pip install -qU langchain_community

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os

# Set a user agent string
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"

In [ ]:
# Importing the WebBaseLoader from langchain_community
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/110690555461018310-software-engineer-iii-infrastructure-core")

In [11]:
page_data = loader.load().pop().page_content
print(page_data)

Software Engineer III, Infrastructure, Core — Google CareersCareersCareersSkip navigation linkshomehomeHomeHomework_outlinework_outlineJobsJobsnoogler_hatnoogler_hatStudentsStudentsgooglegoogleHow we workHow we workhandymanhandymanHow we hireHow we hireperson_outlineperson_outlineYour careerYour careerhelp_outlineHelp linkfeedbackSend feedbackmore_vert HelpSend FeedbackSign inCareersCareershomeHomework_outlineJobsexpand_morenoogler_hatStudentsexpand_moregoogleHow we workexpand_morehandymanHow we hireexpand_moreperson_outlineYour careerexpand_morejob detailsarrow_backBack to jobs searchJobs search results2,782  jobs matchedSoftware Engineer III, Infrastructure, CoreBengaluru, Karnataka, IndiaSenior Software Engineer, Android PartnerWarsaw, PolandSenior Software Engineer, Google Cloud DataprocSunnyvale, CA, USASoftware Engineer III, Google CloudBengaluru, Karnataka, India; Pune, Maharashtra, India; +2 more; +1 moreAccount Manager, Large Customer Sales (English, Thai)Bangkok, ThailandCust

using promt technique

In [ ]:
from langchain_core.prompts import PromptTemplate
# text extraction promt Template
promt_extract = PromptTemplate.from_template(
    '''
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION
    The scraped text id from the carrer's page of a website.
    your job is to extract the job posting and return them in JSON format containing following keys: 'role', experience', 'skills', and 'description'.
    only return the valid JSON. make sure no other key present insise the key-value pair.
    ### VALID JSON (NO PREAMBLE):
    '''
)

In [ ]:
# Using the prompt template with the loaded page content
chain_extract = promt_extract | llm # chian formation
res = chain_extract.invoke(
    input ={
        'page_data': page_data
    }
)


str

In [19]:
# print(res.content)
type(res.content)

str

In [23]:
#  convert str into json object (res.content)
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer III, Infrastructure, Core',
 'experience': ['Bachelor’s degree or equivalent practical experience.',
  '2 years of experience with software development in one or more programming languages, or 1 year of experience with an advanced degree.',
  "Master's degree or PhD in Computer Science or related technical fields.",
  '2 years of experience building and developing infrastructure, distributed systems or networks, or experience with compute technologies, storage, or hardware architecture.',
  '2 years of experience with data structures or algorithms in either an academic or industry setting.',
  'Experience developing accessible technologies.'],
 'skills': ['Software development',
  'Infrastructure development',
  'Distributed systems',
  'Compute technologies',
  'Storage',
  'Hardware architecture',
  'Data structures',
  'Algorithms',
  'Accessible technologies'],
 'description': "Google's software engineers develop the next-generation technologies that cha

In [27]:
type(json_res) # dict

dict

### my skills data
store my skills with portfolio links on chroma db

In [46]:

import pandas as pd
df = pd.read_csv('my_portfolio.csv')
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [47]:
import chromadb
# Initialize the ChromaDB client and create a collection
client = chromadb.PersistentClient('vectorstore') # persistant Client create a db on local disk
# 
collection = client.get_or_create_collection(name="portfolio")

In [48]:
import uuid

# / if present continue else iterate through all the portfolio csv
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                                     metadatas={"links": row["Links"]},
                                     ids=[str(uuid.uuid4())])
        
# vectorstore is created

In [67]:

links = collection.query(query_texts=["Experience in python", "Experience in react"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [68]:
# links = collection.query(query_texts=["Experience in python", "Experience in react"], n_results=2).get('metadatas', [])
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [69]:
job

{'role': 'Software Engineer III, Infrastructure, Core',
 'experience': ['Bachelor’s degree or equivalent practical experience.',
  '2 years of experience with software development in one or more programming languages, or 1 year of experience with an advanced degree.',
  "Master's degree or PhD in Computer Science or related technical fields.",
  '2 years of experience building and developing infrastructure, distributed systems or networks, or experience with compute technologies, storage, or hardware architecture.',
  '2 years of experience with data structures or algorithms in either an academic or industry setting.',
  'Experience developing accessible technologies.'],
 'skills': ['Software development',
  'Infrastructure development',
  'Distributed systems',
  'Compute technologies',
  'Storage',
  'Hardware architecture',
  'Data structures',
  'Algorithms',
  'Accessible technologies'],
 'description': "Google's software engineers develop the next-generation technologies that cha

In [70]:
job = json_res
job['skills']

['Software development',
 'Infrastructure development',
 'Distributed systems',
 'Compute technologies',
 'Storage',
 'Hardware architecture',
 'Data structures',
 'Algorithms',
 'Accessible technologies']

In [78]:
# Promt tempelate
# {providing argument in tempelate}
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        you are Ayush, a Software developer Engineer, working at some company. ayush, a passionate and curious learner diving deep into tech from the ground up. I started with C++ and problem-solving, and now I’m actively building full-stack AI web apps and exploring machine learning projects in NLP and computer vision. 
        I’m currently preparing for placements, with a strong focus on DSA, project development, and real-world impact through AI. I constantly seek opportunities to collaborate, learn, and grow in the tech ecosystem.
        ayush was tech co-lead of google developer group.
        Your job is to write a cold email to the recruter regarding the job mentioned above describing the capability of ayush in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Scribo's portfolio: {link_list}
        Remember you are Ayush, BDE at Scribo. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

# invoke chain
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})

In [79]:
print(res.content)

<think>
Okay, I need to help Ayush write a cold email to a recruiter for the Software Engineer III position at Google. Let me start by understanding the job description and Ayush's background.

The job requires a bachelor's degree or equivalent experience, 2 years in software development, experience with infrastructure, distributed systems, data structures, algorithms, and accessible technologies. Skills include software and infrastructure development, distributed systems, compute tech, storage, hardware architecture, data structures, algorithms, and accessible tech.

Ayush is a software developer with a background in C++ and problem-solving. He's now into full-stack AI web apps and ML projects in NLP and CV. He's preparing for placements, focusing on DSA, projects, and AI impact. He was the tech co-lead of the Google Developer Group.

I need to highlight Ayush's relevant skills and experiences. He has experience in software development, which aligns with the job. His work in AI and ML